In [13]:
# Imports

import pandas as pd
import os

In [14]:
# Funções auxiliares

def verificarSeSaoTodosNumeros(dataFramePNumerico):
    todos_numeros = pd.to_numeric(dataFramePNumerico['ano'], errors='coerce').notna().all()
    print(f'Todos os valores na coluna "ano" são números? {todos_numeros}')

def ler_csv_para_dataframe(caminho_arquivo_csv):
    df = open(caminho_arquivo_csv, 'r', newline='', encoding='utf-8')
    return df
    
def renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova):
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo_antigo = os.path.join(pasta, nome_arquivo)
        if palavra_antiga in nome_arquivo:
            novo_nome_arquivo = nome_arquivo.replace(palavra_antiga, palavra_nova)
            caminho_arquivo_novo = os.path.join(pasta, novo_nome_arquivo)
            os.rename(caminho_arquivo_antigo, caminho_arquivo_novo)
    print("Arquivos renomeados com sucesso!")

def obter_nomes_apos_substring(pasta):
    nomes_extraidos = []
    for nome_arquivo in os.listdir(pasta):
        if "Scrapped_Car_Review_" in nome_arquivo and nome_arquivo.endswith(".csv"):
            parte_nome = nome_arquivo.split("Scrapped_Car_Review_")[1].replace(".csv", "")
            nomes_extraidos.append(parte_nome)
    return nomes_extraidos

In [15]:
# Tratamento dos arquivos nas pastas

pasta = './Data/'  # Substitua pelo caminho da sua pasta
palavra_antiga = "Reviews"
palavra_nova = "Review"

renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

Arquivos renomeados com sucesso!


In [16]:
# Upload dos datasets

marcas_de_carro = obter_nomes_apos_substring("./Data/")

for marca in marcas_de_carro:
    file_path = f'./Data/Scrapped_Car_Review_{marca}.csv'
    df = ler_csv_para_dataframe(file_path)
    print(df)
    # print(f'O DataFrame para a marca {marca} tem {df.shape} colunas.')

    # if file_path == './Data/Scrapped_Car_Review_AMGeneral.csv':
    #     df.head()



<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_Honda.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_Bentley.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_Cadillac.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_AlfaRomeo.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_AstonMartin.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_Buick.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_Volkswagen.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_AMGeneral.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_Bugatti.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_GMC.csv' mode='r' encoding='utf-8'>
<_io.TextIOWrapper name='./Data/Scrapped_Car_Review_BMW.csv' mode='r' encodi